In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from enum import Enum

prefix = "MSFT_2012-06-21_34200000_57600000"

trades_data_file = f"./data/{prefix}_message_5.csv"
orderbook_data_file = f"./data/{prefix}_orderbook_5.csv"


class Direction(Enum):
    BUY = 1
    SELL = -1


class OrderType(Enum):
    SUBMISSION = 1
    CANCELLATION = 2
    DELETION = 3
    EXECUTION_VISIBLE = 4
    EXECUTION_HIDDEN = 5
    TRADING_HALT = 7


trade_df = pd.read_csv(trades_data_file, names=(
    'timestamp',
    'type',
    'orderId',
    'size',
    'trade_price',
    'direction'
))

orderbook_df = pd.read_csv(orderbook_data_file, names=(
    'ask_1_price',
    'ask_1_size',
    'bid_1_price',
    'bid_1_size',

    'ask_2_price',
    'ask_2_size',
    'bid_2_price',
    'bid_2_size',

    'ask_3_price',
    'ask_3_size',
    'bid_3_price',
    'bid_3_size',

    'ask_4_price',
    'ask_4_size',
    'bid_4_price',
    'bid_4_size',

    'ask_5_price',
    'ask_5_size',
    'bid_5_price',
    'bid_5_size',
))

df = pd.concat([trade_df, orderbook_df], axis=1, )
df['timestamp'] = df['timestamp'] - 34200
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.set_index('timestamp')

df = df.iloc[:20_000]

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

# display(df.head(10))

# Feature Engineering

- Midprice
- Imbalance
- Sum Trade 1s, 3s, 5s
- (bid/ask) advance time
- last trade time

In [47]:
df['midprice'] = (df['ask_1_price'] + df['bid_1_price']) / 2
df['imbalance'] = df['ask_1_size'] - df['bid_1_size'] / (df['ask_1_size'] + df['bid_1_size'])


# rolling_df_depth = 100
# def sum_within_1_second(row):
#     prior_idx = max(row.name - rolling_df_depth, 0)  
#     rolling_df = df.iloc[prior_idx:row.name]
    
#     mask = rolling_df['type'] == OrderType.EXECUTION_VISIBLE.value & \
#         (rolling_df['timestamp'] >= row['timestamp'] - pd.Timedelta(seconds=1)) & \
#         (rolling_df['timestamp'] <= row['timestamp'])
    
#     sum = rolling_df[mask]['size'].sum()
#     return sum

# df['sum_within_1s'] = df.apply(sum_within_1_second, axis=1)

executionOrdersMask = df['type'] == OrderType.EXECUTION_VISIBLE.value

# sum_within_1s = df[executionOrdersMask]['size'].rolling(window=pd.Timedelta(seconds=1)).sum()
df['sum_within_1s'] = df['size'].where(df['type'] == OrderType.EXECUTION_VISIBLE.value).rolling(window=pd.Timedelta(seconds=1)).sum().fillna(0)

# newdf = df[executionOrdersMask]
# newdf['sum_within_1s'] = sum_within_1s

# display(newdf)
display(df)

,type,orderId,size,trade_price,direction,ask_1_price,ask_1_size,bid_1_price,bid_1_size,ask_2_price,ask_2_size,bid_2_price,bid_2_size,ask_3_price,ask_3_size,bid_3_price,bid_3_size,ask_4_price,ask_4_size,bid_4_price,bid_4_size,ask_5_price,ask_5_size,bid_5_price,bid_5_size,midprice,imbalance,sum_within_1s
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.013994120,3,16085616,100,310400,-1,309900,3788,309500,300,310500,100,309300,3986,310600,100,309200,100,310700,200,309100,300,310800,200,308900,100,309700.0,3787.926614,0.0
1970-01-01 00:00:00.013994120,1,16116348,100,310500,-1,309900,3788,309500,300,310500,200,309300,3986,310600,100,309200,100,310700,200,309100,300,310800,200,308900,100,309700.0,3787.926614,0.0
1970-01-01 00:00:00.015247805,1,16116658,100,310400,-1,309900,3788,309500,300,310400,100,309300,3986,310500,200,309200,100,310600,100,309100,300,310700,200,308900,100,309700.0,3787.926614,0.0
1970-01-01 00:00:00.015442111,1,16116704,100,310500,-1,309900,3788,309500,300,310400,100,309300,3986,310500,300,309200,100,310600,100,309100,300,310700,200,308900,100,309700.0,3787.926614,0.0
1970-01-01 00:00:00.015789148,1,16116752,100,310600,-1,309900,3788,309500,300,310400,100,309300,3986,310500,300,309200,100,310600,200,309100,300,310700,200,308900,100,309700.0,3787.926614,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:07:00.186062800,3,25378160,600,310400,1,310600,4900,310400,4460,310700,5400,310300,4400,310800,6300,310200,3700,310900,3933,310100,4600,311000,3500,310000,53400,310500.0,4899.523504,2201.0
1970-01-01 00:07:00.186065848,1,25378167,600,310400,1,310600,4900,310400,5060,310700,5400,310300,4400,310800,6300,310200,3700,310900,3933,310100,4600,311000,3500,310000,53400,310500.0,4899.491968,2201.0
1970-01-01 00:07:00.186119702,3,25378163,600,310400,1,310600,4900,310400,4460,310700,5400,310300,4400,310800,6300,310200,3700,310900,3933,310100,4600,311000,3500,310000,53400,310500.0,4899.523504,2201.0
